In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
from torchtext.legacy import datasets
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy

# Preprocess the text data
def preprocess_text(tokens):
    
    # Convert to lowercase and remove non-alphabetic characters
    words = [word.lower() for word in tokens if word.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    clean_words = [word for word in words if word not in stop_words]
    
    return clean_words

# Create the dataset
TEXT = data.Field(
    tokenize='spacy',
    preprocessing=preprocess_text,
    lower=True,
    batch_first=True)

LABEL = data.LabelField(dtype=torch.float, batch_first=True)

dataset = data.TabularDataset(
    path="data.csv",
    format="csv",
    fields={
        "text": ("text", TEXT),
        "ret_label": ("label", LABEL)
    },
)

/Users/olliesun/opt/anaconda3/envs/NLP2/lib/python3.8/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [2]:
# Split the dataset into training and testing sets
train_data, test_data = dataset.split(split_ratio=0.8)

# Build the vocabulary
TEXT.build_vocab(train_data, max_size=10000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

# Create data iterators
BATCH_SIZE = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True
)

# Create the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        lstm_output, (hidden, cell) = self.lstm(embedded)
        output = self.fc(self.dropout(lstm_output[:,-1,:]))
        return output

In [5]:
# Instantiate the model
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 64
OUTPUT_DIM = 1
N_LAYERS = 2
DROPOUT = 0.5

model = LSTMClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                       N_LAYERS, DROPOUT).to(device)

# Train the model
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


N_EPOCHS = 10

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    print(
        f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%'
    )

Epoch: 01, Train Loss: 0.696, Train Acc: 49.07%, Val. Loss: 0.692, Val. Acc: 66.67%
Epoch: 02, Train Loss: 0.687, Train Acc: 56.02%, Val. Loss: 0.693, Val. Acc: 66.67%
Epoch: 03, Train Loss: 0.693, Train Acc: 51.39%, Val. Loss: 0.692, Val. Acc: 66.67%
Epoch: 04, Train Loss: 0.694, Train Acc: 57.87%, Val. Loss: 0.694, Val. Acc: 66.67%
Epoch: 05, Train Loss: 0.686, Train Acc: 60.19%, Val. Loss: 0.691, Val. Acc: 66.67%
Epoch: 06, Train Loss: 0.689, Train Acc: 56.48%, Val. Loss: 0.690, Val. Acc: 66.67%
Epoch: 07, Train Loss: 0.696, Train Acc: 48.61%, Val. Loss: 0.685, Val. Acc: 62.50%
Epoch: 08, Train Loss: 0.703, Train Acc: 48.15%, Val. Loss: 0.683, Val. Acc: 62.50%
Epoch: 09, Train Loss: 0.700, Train Acc: 54.17%, Val. Loss: 0.678, Val. Acc: 62.50%
Epoch: 10, Train Loss: 0.692, Train Acc: 54.17%, Val. Loss: 0.678, Val. Acc: 62.50%
